unzip json files in to a folder in google drive cloud

---



In [36]:
# from zipfile import ZipFile
# with ZipFile('drive/MyDrive/datasets.zip', 'r') as zipObj:
  # zipObj.extractall('drive/MyDrive/test')

In [37]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)

In [38]:
path = '/content/drive/MyDrive/test/Dataset Yelp/'

In [39]:
df_business = pd.read_json(f'{path}business.json', lines=True)

In [40]:
business_index = pd.DataFrame()
business_index['id'] = list(range(1, len(df_business) + 1, 1))
business_index['business_id'] = df_business['business_id'].copy()
business_index.head()

,id,business_id
0,1,Pns2l4eNsfO8kk83dixA6A
1,2,mpf3x-BjTdTEA3yCZrAYPw
2,3,tUFrWirKiKi_TAnsVWINQQ
3,4,MTSW4McQd7CbVtyjqoe9mw
4,5,mWMc6_wTdE0EUBKIGXDVfA


In [41]:
df_business.drop(['business_id'], axis=1, inplace=True)
new_col = list(range(1, len(df_business) +1, 1))
df_business.insert(loc = 0, column = 'business_id', value = new_col) 

In [42]:
df_business['city'] = df_business['city'].apply(lambda x: x.lower())
df_business['city'] = df_business['city'].str.replace(',','')
df_business['city'] = df_business['city'].str.replace('.','')
df_business['city'] = df_business['city'].apply(lambda x: x.strip())
df_business['city'] = df_business['city'].apply(lambda x: " ".join(x.split()))
df_business['city'] = df_business['city'].apply(lambda x: x.title())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
df_business['categories'] = df_business['categories'].fillna('0')

In [44]:
cat_list = ['Restaurants', 'Home Services', 'Auto Services', 'Auto Repair', 'Beauty & Spas', 'Food', 'Nightlife', 'Hotels', 'Local Services', 'Pets']
def validator(value):
  if value in cat_list:
    return 1000
  else:
    return 0
print(validator('Restaurants'))

def cat_to_list(value):
  value = value.split(', ')
  value.sort(key=validator, reverse=True)
  return ', '.join(value)

  
df_business['categories'] = df_business['categories'].apply(lambda x: cat_to_list(x))

1000


In [45]:
# df[df['A'].str.contains("hello")]
# df_business['categories'][df_business['categories'].str.contains('Restaurant')]
# df.sort_values(by=['col1'])
city_dict = {'Belle Chase': 'Belle Chasse',
             'Abington Township': 'Abington',
             'Ashland City': 'Ashland',
             'Bellefontaine Neighbors': 'Bellefontaine',
             'Bellville': 'Belleville',
             'Belleair Blf': 'Belleair Bluffs',
             'Bethel Township': 'Bethel',
             'Bensalem Pa': 'Bensalem',
             'Bensalem Township': 'Bensalem',
             'Boise City': 'Boise',
             'Burlington Township': 'Burlington',
             "Carney'S Point": 'Carneys Point',
             'Cedar Brook': 'Cedarbrook',
             '/': '',
             'Conshohoeken':'Conshohocken',
             'Delran Township': 'Delran',
             'Delran Twp': 'Delran',
             'Concord Township': 'Concord',
             'Deptford Township': 'Deptford',
             'Eastampton Township': 'Eastampton',
             ' Township': '',
             'Fairview Hts': 'Fairview',
             '-': '',
             ' City': '',
             ' Twp': '',
             'Bch': 'Beach',
             'Land O Lakes':"Land O'Lakes",
             "Land O' Lakes":"Land O'Lakes",
             'Mccordsville': 'Mc Cordsville',
             'Metarie': 'Metairie',
             'Mt Laurel Twp Nj': 'Mt Laurel',
             'Sqaure': 'Square',
             'O Fallon': "O'Fallon",
             "O' Fallon": "O'Fallon",
             'Phila': 'Philadelphia',
             'Philadephia': 'Philadelphia',
             'Philly': 'Philadelphia',
             'Redingtn Shor': 'Redington Shore',
             'Redington Shores': 'Redington Shore',
             'Riverview Fl': 'Riverview',
             'Saintt': 'Saint',
             'Tierre': 'Tierra',
             "Town 'N' Country": 'Town & Country',
             'Town And Country': 'Town & Country',
             'Town N Country': 'Town & Country',
             'Tuscon': 'Tucson'
             }

df_business = df_business.replace({'city': city_dict})
df_restaurants = df_business[df_business['categories'].str.contains('Restaurant')]
city_restaurants = df_restaurants.sort_values(by=['city'])

In [46]:
df_business['categories'].isin(['Restaurant']).value_counts()

False    150346
Name: categories, dtype: int64

In [47]:
df_city_state = df_business[['city', 'state']].drop_duplicates().copy()
new_col2 = list(range(1, len(df_city_state)+1, 1))
df_city_state.insert(loc = 0, column = 'city_state_id', value = new_col2) 
df_city_state

,city_state_id,city,state
0,1,Santa Barbara,CA
1,2,Affton,MO
2,3,Tucson,AZ
3,4,Philadelphia,PA
4,5,Green Lane,PA
...,...,...,...
149142,1209,Woodbury Hts,NJ
149540,1210,Slidell,LA
149841,1211,Wales,PA
150054,1212,Upper Moreland,PA


In [48]:
# redondear latitud y longitud para reducir ciudades
import math
math_aux = 100
df_business['latitude']  = df_business['latitude'].apply(lambda x: math.floor(x*math_aux)/math_aux)
df_business['longitude'] = df_business['longitude'].apply(lambda x: math.floor(x*math_aux)/math_aux)

In [49]:
df_business['city'].drop_duplicates().tolist().sort()

In [50]:
def cat_to_col(value):
  aux_dict = {}
  x = value.split(', ')
  for i in x:
    aux_dict[i] = 1
  return aux_dict

In [51]:
def dict_to_columns(df_i, column):
  df_i[column] = df_i[column].fillna('{}')
  df_i.reset_index(inplace=True)
  df_o = df_i.join(pd.json_normalize(df_i.pop(column)))
  if 'level_0' in df_o.columns:
    df_o.drop(['level_0'], axis=1, inplace=True)

  return df_o

In [52]:
# funcion para tratar la columna atributos y obtener solo valores verdaderos del diccionario atributos
def attr_to_list(i):
  attributes_list = []
  try:
    for j in i:
      if i[j] == 'True' or i[j] == '1' or i[j] == "u'free'":
        attributes_list.append(j)
    return attributes_list
  except:
    return []

In [53]:
df_restaurants['attributes'] = df_restaurants['attributes'].apply(lambda x: attr_to_list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"[BikeParking, RestaurantsPriceRange2, Restaura...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,6,Sonic Drive-In,615 S Main St,Ashland,TN,37015,36.269593,-87.058943,2.0,6,1,"[BusinessAcceptsCreditCards, OutdoorSeating, R...","Food, Restaurants, Burgers, Fast Food, Sandwic...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,9,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"[Caters, RestaurantsTakeOut, HasTV, BusinessAc...","Restaurants, Nightlife, Pubs, Italian, Bars, A...",None
9,10,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"[BusinessAcceptsCreditCards, OutdoorSeating, G...","Restaurants, Food, Ice Cream & Frozen Yogurt, ...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,12,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,[],"Food, Restaurants, Vietnamese, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [55]:
df_business['attributes'] = df_business['attributes'].apply(lambda x: attr_to_list(x))

In [56]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.42,-119.72,5.0,7,0,[ByAppointmentOnly],"Doctors, Traditional Chinese Medicine, Naturop...",None
1,2,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.55,-90.34,3.0,15,1,[BusinessAcceptsCreditCards],"Local Services, Shipping Centers, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,3,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.22,-110.89,3.5,22,0,"[BikeParking, BusinessAcceptsCreditCards, Whee...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.95,-75.16,4.0,80,1,"[BikeParking, RestaurantsPriceRange2, Restaura...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,5,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.33,-75.48,4.5,13,1,"[BusinessAcceptsCreditCards, WheelchairAccessi...","Food, Brewpubs, Breweries","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [57]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [58]:
import datetime

now = datetime.datetime.now()
print(now.strftime("%A"))
print(now)

Tuesday
2022-11-22 14:47:49.927894


In [59]:
df_business.to_parquet('/content/drive/MyDrive/output/business.parquet.gzip', compression='gzip')
df_restaurants.to_parquet('/content/drive/MyDrive/output/restaurants.parquet.gzip', compression='gzip')
business_index.to_csv('/content/drive/MyDrive/output/business_index.csv', index=False)

In [60]:
# import pickle

# with open("/content/drive/MyDrive/index.pickle", "wb") as f:
#     pickle.dump(max_index, f)

In [61]:
import pickle

with open("/content/drive/MyDrive/index.pickle", "rb") as f:
    max_index = pickle.load(f)